In [ ]:
import numpy as np
from math import sqrt
import csv

def Get_All_Data(TG,time_lag,TG_in_one_day,forecast_day_number,TG_in_one_week):
	metro_enter = []
	with open('in_'+str(TG)+'min.csv') as f:
		data = csv.reader(f, delimiter=",")
		for line in data:
			line=[int(x) for x in line]
			metro_enter.append(line)

	def get_train_data_enter(data,time_lag,TG_in_one_day,forecast_day_number,TG_in_one_week):
		data = np.array(data)
		data2 = np.zeros((data.shape[0], data.shape[1]))
		a = np.max(data)
		b = np.min(data)
		for i in range(len(data)):
			for j in range(len(data[0])):
				data2[i, j] = round((data[i, j]-b)/(a-b), 5)

		X_train_1 = [[] for i in range(TG_in_one_week, len(data2[0]) - time_lag+1 - TG_in_one_day*forecast_day_number)]
		Y_train = []
		for index in range(TG_in_one_week, len(data2[0]) - time_lag+1 - TG_in_one_day*forecast_day_number):
			for i in range(276):
				temp=data2[i,index-TG_in_one_week: index + time_lag-1-TG_in_one_week].tolist()
				temp.extend(data2[i,index-TG_in_one_day: index + time_lag-1-TG_in_one_day])
				temp.extend(data2[i,index: index + time_lag-1])
				X_train_1[index-TG_in_one_week].append(temp)
			Y_train.append(data2[:,index + time_lag-1])
		X_train_1,Y_train = np.array(X_train_1), np.array(Y_train)
		print(X_train_1.shape,Y_train.shape)

		X_test_1 = [[] for i in range(len(data2[0]) - TG_in_one_day*forecast_day_number,len(data2[0])-time_lag+1)]
		Y_test = []
		for index in range(len(data2[0]) - TG_in_one_day*forecast_day_number,len(data2[0])-time_lag+1):
			for i in range(276):
				temp = data2[i, index-TG_in_one_week: index + time_lag-1-TG_in_one_week].tolist()
				temp.extend(data2[i, index-TG_in_one_day: index + time_lag-1-TG_in_one_day])
				temp.extend(data2[i, index: index + time_lag-1])
				X_test_1[index-(len(data2[0]) - TG_in_one_day*forecast_day_number)].append(temp)
			Y_test.append(data2[:, index + time_lag-1])
		X_test_1,Y_test = np.array(X_test_1), np.array(Y_test)
		print(X_test_1.shape, Y_test.shape)

		Y_test_original = []
		for index in range(len(data[0]) - TG_in_one_day*forecast_day_number,len(data[0])-time_lag+1):
			Y_test_original.append(data[:, index + time_lag-1])
		Y_test_original = np.array(Y_test_original)

		print(Y_test_original.shape)

		return X_train_1,Y_train,X_test_1,Y_test,Y_test_original,a,b


	X_train_1,Y_train,X_test_1,Y_test,Y_test_original,a,b=get_train_data_enter(metro_enter,time_lag,TG_in_one_day,forecast_day_number,TG_in_one_week)
	print(a,b)


	metro_exit = []
	with open('out_'+str(TG)+'min.csv') as f:
		data = csv.reader(f, delimiter=",")
		for line in data:
			line = [int(x) for x in line]
			metro_exit.append(line)

	def get_train_data_exit(data,time_lag,TG_in_one_day,forecast_day_number,TG_in_one_week):
		data = np.array(data)
		data2 = np.zeros((data.shape[0], data.shape[1]))
		a = np.max(data)
		b = np.min(data)
		for i in range(len(data)):
			for j in range(len(data[0])):
				data2[i, j]=round((data[i, j]-b)/(a-b), 5)
		X_train_1 = [[] for i in range(TG_in_one_week, len(data2[0]) - time_lag+1 - TG_in_one_day*forecast_day_number)]
		for index in range(TG_in_one_week, len(data2[0]) - time_lag+1 - TG_in_one_day*forecast_day_number):
			for i in range(276):
				temp=data2[i, index-TG_in_one_week: index + time_lag-1-TG_in_one_week].tolist()
				temp.extend(data2[i, index-TG_in_one_day: index + time_lag-1-TG_in_one_day])
				temp.extend(data2[i, index: index + time_lag-1])
				X_train_1[index-TG_in_one_week].append(temp)
		X_train_1 = np.array(X_train_1)
		print(X_train_1.shape)

		X_test_1 = [[] for i in range(len(data2[0]) - TG_in_one_day*forecast_day_number, len(data2[0])-time_lag+1)]
		for index in range(len(data2[0]) - TG_in_one_day*forecast_day_number, len(data2[0])-time_lag+1):
			for i in range(276):
				temp = data2[i,index-TG_in_one_week: index + time_lag-1-TG_in_one_week].tolist()
				temp.extend(data2[i, index-TG_in_one_day: index + time_lag-1-TG_in_one_day])
				temp.extend(data2[i, index: index + time_lag-1])
				X_test_1[index-(len(data2[0]) - TG_in_one_day*forecast_day_number)].append(temp)
		X_test_1 = np.array(X_test_1)
		print(X_test_1.shape)
		return X_train_1, X_test_1

	X_train_2, X_test_2 = get_train_data_exit(metro_exit, time_lag, TG_in_one_day, forecast_day_number, TG_in_one_week)


	adjacency = []
	with open('adjacency.csv') as f:
		data = csv.reader(f, delimiter=",")
		for line in data:
			line = [float(x) for x in line]
			adjacency.append(line)
	adjacency = np.array(adjacency)

	I = np.matrix(np.eye(276))
	A_hat = adjacency+I
	D_hat = np.array(np.sum(A_hat, axis=0))[0]
	D_hat_sqrt = [sqrt(x) for x in D_hat]
	D_hat_sqrt = np.array(np.diag(D_hat_sqrt))
	D_hat_sqrtm_inv = np.linalg.inv(D_hat_sqrt)
	D_A_final = np.dot(D_hat_sqrtm_inv, A_hat)
	D_A_final = np.dot(D_A_final, D_hat_sqrtm_inv)
	print(D_A_final.shape)
	def get_train_data_graph(data,D_A_final,time_lag,TG_in_one_day,forecast_day_number,TG_in_one_week,):
		data = np.array(data)
		data2 = np.zeros((data.shape[0], data.shape[1]))
		a = np.max(data)
		b = np.min(data)
		for i in range(len(data)):
			for j in range(len(data[0])):
				data2[i,j]=round((data[i,j]-b)/(a-b),5)
		X_train_1 = [[] for i in range(TG_in_one_week, len(data2[0]) - time_lag+1 - TG_in_one_day*forecast_day_number)]
		for index in range(TG_in_one_week, len(data2[0]) - time_lag+1 - TG_in_one_day*forecast_day_number):
			for i in range(276):
				temp=data2[i,index: index + time_lag-1]
				X_train_1[index-TG_in_one_week].append(temp)
			X_train_1[index-TG_in_one_week] = np.dot(D_A_final, X_train_1[index-TG_in_one_week])
		X_train_1= np.array(X_train_1)
		print(X_train_1.shape)

		X_test_1 = [[] for i in range(len(data2[0]) - TG_in_one_day*forecast_day_number,len(data2[0])-time_lag+1)]
		for index in range(len(data2[0]) - TG_in_one_day*forecast_day_number,len(data2[0])-time_lag+1):
			for i in range(276):
				temp = data2[i,index: index + time_lag-1]
				X_test_1[index-(len(data2[0]) - TG_in_one_day*forecast_day_number)].append(temp)
			X_test_1[index-(len(data2[0]) - TG_in_one_day*forecast_day_number)] = np.dot(D_A_final, X_test_1[index-(len(data2[0]) - TG_in_one_day*forecast_day_number)])
		X_test_1 = np.array(X_test_1)
		print(X_test_1.shape)

		return X_train_1,X_test_1

	X_train_3, X_test_3 = get_train_data_graph(metro_enter, D_A_final, time_lag, TG_in_one_day, forecast_day_number, TG_in_one_week)


	Weather = []
	with open(str(TG)+'min after normalization.csv') as f:
		data = csv.reader(f, delimiter=",")
		for line in data:
			line = [float(x) for x in line]
			Weather.append(line)

	def get_train_data_weather_PM(data, time_lag, TG_in_one_day, forecast_day_number, TG_in_one_week,):
		data = np.array(data)

		X_train_1 = [[] for i in range(TG_in_one_week, len(data[0]) - time_lag+1 - TG_in_one_day*forecast_day_number)]
		for index in range(TG_in_one_week, len(data[0]) - time_lag+1 - TG_in_one_day*forecast_day_number):
			for i in range(len(data)):

				X_train_1[index-TG_in_one_week].append(data[i,index: index + time_lag-1])
		X_train_1 = np.array(X_train_1)
		print(X_train_1.shape)

		X_test_1 = [[] for i in range(len(data[0]) - TG_in_one_day*forecast_day_number, len(data[0])-time_lag+1)]
		for index in range(len(data[0]) - TG_in_one_day*forecast_day_number, len(data[0])-time_lag+1):
			for i in range(len(data)):
				X_test_1[index-(len(data[0]) - TG_in_one_day*forecast_day_number)].append(data[i, index: index + time_lag-1])
		X_test_1 = np.array(X_test_1)
		print(X_test_1.shape)
		return X_train_1,X_test_1

	X_train_4, X_test_4 = get_train_data_weather_PM(Weather, time_lag, TG_in_one_day, forecast_day_number, TG_in_one_week)

	return X_train_1, Y_train, X_test_1, Y_test, Y_test_original, a, b, X_train_2, X_test_2, X_train_3, X_test_3, X_train_4, X_test_4


In [ ]:
from google.colab import files

# Upload the files
uploaded = files.upload()

Saving metrics.py to metrics.py
Saving 30min after normalization.csv to 30min after normalization.csv
Saving 15min after normalization.csv to 15min after normalization.csv
Saving 10min after normalization.csv to 10min after normalization.csv
Saving out_30min.csv to out_30min.csv
Saving out_15min.csv to out_15min.csv
Saving out_10min.csv to out_10min.csv
Saving in_30min.csv to in_30min.csv
Saving in_15min.csv to in_15min.csv
Saving in_10min.csv to in_10min.csv
Saving adjacency.csv to adjacency.csv


In [ ]:
import os

# List files in the current directory
print(os.listdir())

['.config', 'metrics.py', 'in_30min.csv', 'testresult', 'in_15min.csv', 'out_30min.csv', '10min after normalization.csv', '30min after normalization.csv', 'model.png', '__pycache__', 'adjacency.csv', '15min after normalization.csv', 'in_10min.csv', 'out_10min.csv', 'out_15min.csv', 'sample_data']


# 10 **MINS**

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(2)
import numpy as np
np.set_printoptions(threshold=np.inf)
import time, os
import keras
keras.backend.set_image_data_format('channels_last')
from keras.layers import *
from keras.models import *
from tensorflow.keras.utils import plot_model
from keras.models import load_model
from keras.optimizers import Adam
from metrics import evaluate_performance



os.environ["PATH"] += os.pathsep + 'E:/Program Files (x86)/Graphviz2.38/bin' #used for visualizing the model

global_start_time = time.time()

def Unit(x, filters, pool=False):
	res = x
	if pool:
		x = MaxPooling2D(pool_size=(2, 2), padding="same")(x)
		res = Conv2D(filters=filters, kernel_size=[1, 1], strides=(2, 2), padding="same")(res)
	out = BatchNormalization()(x)
	out = Activation("relu")(out)
	out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

	out = BatchNormalization()(out)
	out = Activation("relu")(out)
	out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

	out = keras.layers.add([res, out])

	return out

def attention_3d_block(inputs,timesteps):
    #input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(timesteps, activation='linear')(a)
    a_probs = Permute((2, 1))(a)
    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

# Define the model
def multi_input_model(time_lag):
    """build multi input model构建多输入模型"""
    input1_ = Input(shape=(276, time_lag-1, 3), name='input1')
    input2_ = Input(shape=(276, time_lag-1, 3), name='input2')
    input3_ = Input(shape=(276, time_lag-1, 1), name='input3')
    input4_ = Input(shape=(11, time_lag-1, 1), name='input4')
    #first input
    x1 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input1_)
    x1 = Unit(x1, 32)
    x1 = Unit(x1, 64, pool=True)
    x1 = Flatten()(x1)
    x1 = Dense(276)(x1)

    # second input
    x2 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input2_)
    x2 = Unit(x2, 32)
    x2 = Unit(x2, 64, pool=True)
    x2 = Flatten()(x2)
    x2 = Dense(276)(x2)

    # third input
    x3 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input3_)
    x3 = Unit(x3, 32)
    x3 = Unit(x3, 64, pool=True)
    x3 = Flatten()(x3)
    x3 = Dense(276)(x3)

    # fourth input
    x4 = Flatten()(input4_)
    x4 = Dense(276)(x4)
    x4 = Reshape(target_shape=(276, 1))(x4)
    x4 = LSTM(128, return_sequences=True, input_shape=(276, 1))(x4)
    x4 = LSTM(276, return_sequences=False)(x4)
    x4 = Dense(276)(x4)

    out = keras.layers.add([x1, x2, x3, x4])
    out = Reshape(target_shape=(276, 1))(out)
    out = LSTM(128, return_sequences=True,input_shape=(276, 1))(out)
    out = attention_3d_block(out, 276)#shape of the output is（276，128）
    out = Flatten()(out)
    out = Dense(276)(out)

    model = Model(inputs=[input1_, input2_, input3_,input4_], outputs=[out]) #[input1_, input2_, input3_]
    return model

def build_model(X_train_1,X_train_2,X_train_3,X_train_4,Y_train,X_test_1,X_test_2,X_test_3,X_test_4,Y_test,\
	Y_test_original,batch_size,epochs,a,time_lag):

	X_train_1 = X_train_1.reshape(X_train_1.shape[0],  276, time_lag-1, 3)
	X_train_2 = X_train_2.reshape(X_train_2.shape[0],  276, time_lag-1, 3)
	X_train_3 = X_train_3.reshape(X_train_3.shape[0],  276, time_lag-1, 1)
	X_train_4 = X_train_4.reshape(X_train_4.shape[0],  11, time_lag-1, 1)
	Y_train = Y_train.reshape(Y_train.shape[0], 276)

	X_test_1 = X_test_1.reshape(X_test_1.shape[0],  276, time_lag-1, 3)
	X_test_2 = X_test_2.reshape(X_test_2.shape[0],  276, time_lag-1, 3)
	X_test_3 = X_test_3.reshape(X_test_3.shape[0],  276, time_lag-1, 1)
	X_test_4 = X_test_4.reshape(X_test_4.shape[0],  11, time_lag-1, 1)
	Y_test = Y_test.reshape(Y_test.shape[0], 276)

	if epochs == 50:
		model = multi_input_model(time_lag)
		model.compile(optimizer=Adam(), loss='mse', metrics=['mse'])
		model.fit([X_train_1, X_train_2, X_train_3, X_train_4], Y_train, batch_size=batch_size, epochs=epochs, verbose=2, shuffle=False)#, validation_split=0.05
		output = model.predict([X_test_1, X_test_2, X_test_3, X_test_4], batch_size=batch_size)
	else:
		# train models every 10 epoches
		model = load_model('testresult/'+str(epochs-10)+'-model-with-graph.h5')
		model.fit([X_train_1, X_train_2, X_train_3, X_train_4], Y_train, batch_size=batch_size, epochs=10, verbose=2, shuffle=False)# , validation_split=0.05
		output = model.predict([X_test_1, X_test_2, X_test_3, X_test_4], batch_size=batch_size)


	predictions = np.zeros((output.shape[0], output.shape[1]))
	for i in range(len(predictions)):
		for j in range(len(predictions[0])):
			predictions[i, j] = round(output[i, j]*a, 0)
			if predictions[i, j] < 0:
				predictions[i, j] = 0

	RMSE,R2,MAE,WMAPE=evaluate_performance(Y_test_original,predictions)
	#visualize the model structure
	plot_model(model, to_file='model.png', show_shapes=True)
	#print(model.summary())

	return model,Y_test_original,predictions,RMSE,R2,MAE,WMAPE

def Save_Data(path,model,Y_test_original,predictions,RMSE,R2,MAE,WMAPE,Run_epoch):
	print(Run_epoch)
	RMSE_ALL=[]
	R2_ALL=[]
	MAE_ALL=[]
	WMAPE_ALL=[]
	Average_train_time=[]
	RMSE_ALL.append(RMSE)
	R2_ALL.append(R2)
	MAE_ALL.append(MAE)
	WMAPE_ALL.append(WMAPE)
	model.save(path+str(Run_epoch)+'-model-with-graph.h5')
	np.savetxt(path+str(Run_epoch)+'-RMSE_ALL.txt', RMSE_ALL)
	np.savetxt(path+str(Run_epoch)+'-R2_ALL.txt', R2_ALL)
	np.savetxt(path+str(Run_epoch)+'-MAE_ALL.txt', MAE_ALL)
	np.savetxt(path+str(Run_epoch)+'-WMAPE_ALL.txt', WMAPE_ALL)
	with open(path+str(Run_epoch)+'-predictions.csv', 'w') as file:
		predictions = predictions.tolist()
		for i in range(len(predictions)):
			file.write(str(predictions[i]).replace("'", "").replace("[", "").replace("]", "")+"\n")
	with open(path+str(Run_epoch)+'-Y_test_original.csv', 'w') as file:
		Y_test_original = Y_test_original.tolist()
		for i in range(len(Y_test_original)):
			file.write(str(Y_test_original[i]).replace("'", "").replace("[", "").replace("]", "")+"\n")
	duration_time = time.time() - global_start_time
	Average_train_time.append(duration_time)
	np.savetxt(path+str(Run_epoch)+'-Average_train_time.txt', Average_train_time)
	print('total training time(s):', duration_time)

X_train_1,Y_train,X_test_1,Y_test,Y_test_original,a,b,X_train_2,X_test_2,X_train_3,X_test_3,X_train_4,X_test_4=\
	Get_All_Data(TG=10, time_lag=6, TG_in_one_day=108, forecast_day_number=5, TG_in_one_week=540)
Run_epoch = 50
for i in range(15):
	model, Y_test_original, predictions, RMSE, R2, MAE, WMAPE = build_model(X_train_1,X_train_2,X_train_3,X_train_4,Y_train,X_test_1,X_test_2,X_test_3,X_test_4,Y_test,\
		Y_test_original,batch_size=64,epochs=Run_epoch,a=a,time_lag=6)
	Save_Data("testresult/", model, Y_test_original, predictions, RMSE, R2, MAE, WMAPE, Run_epoch)
	Run_epoch += 10


(1615, 276, 15) (1615, 276)
(535, 276, 15) (535, 276)
(535, 276)
3188 0
(1615, 276, 15)
(535, 276, 15)
(276, 276)
(1615, 276, 5)
(535, 276, 5)
(1615, 11, 5)
(535, 11, 5)
Epoch 1/50
26/26 - 25s - loss: 0.0557 - mse: 0.0557 - 25s/epoch - 966ms/step
Epoch 2/50
26/26 - 2s - loss: 0.0046 - mse: 0.0046 - 2s/epoch - 94ms/step
Epoch 3/50
26/26 - 2s - loss: 8.6337e-04 - mse: 8.6337e-04 - 2s/epoch - 94ms/step
Epoch 4/50
26/26 - 2s - loss: 3.6808e-04 - mse: 3.6808e-04 - 2s/epoch - 95ms/step
Epoch 5/50
26/26 - 2s - loss: 2.7152e-04 - mse: 2.7152e-04 - 2s/epoch - 95ms/step
Epoch 6/50
26/26 - 2s - loss: 2.3229e-04 - mse: 2.3229e-04 - 2s/epoch - 94ms/step
Epoch 7/50
26/26 - 2s - loss: 2.1374e-04 - mse: 2.1374e-04 - 2s/epoch - 96ms/step
Epoch 8/50
26/26 - 2s - loss: 1.9051e-04 - mse: 1.9051e-04 - 2s/epoch - 94ms/step
Epoch 9/50
26/26 - 2s - loss: 1.9498e-04 - mse: 1.9498e-04 - 2s/epoch - 95ms/step
Epoch 10/50
26/26 - 2s - loss: 1.6850e-04 - mse: 1.6850e-04 - 2s/epoch - 95ms/step
Epoch 11/50
26/26 - 2s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 178.64544486999512
Epoch 1/10
26/26 - 15s - loss: 9.0464e-05 - mse: 9.0464e-05 - 15s/epoch - 558ms/step
Epoch 2/10
26/26 - 3s - loss: 8.6018e-05 - mse: 8.6018e-05 - 3s/epoch - 98ms/step
Epoch 3/10
26/26 - 3s - loss: 8.6693e-05 - mse: 8.6693e-05 - 3s/epoch - 98ms/step
Epoch 4/10
26/26 - 3s - loss: 8.2573e-05 - mse: 8.2573e-05 - 3s/epoch - 99ms/step
Epoch 5/10
26/26 - 3s - loss: 8.1827e-05 - mse: 8.1827e-05 - 3s/epoch - 99ms/step
Epoch 6/10
26/26 - 3s - loss: 7.9653e-05 - mse: 7.9653e-05 - 3s/epoch - 100ms/step
Epoch 7/10
26/26 - 3s - loss: 7.9641e-05 - mse: 7.9641e-05 - 3s/epoch - 100ms/step
Epoch 8/10
26/26 - 3s - loss: 7.9392e-05 - mse: 7.9392e-05 - 3s/epoch - 100ms/step
Epoch 9/10
26/26 - 3s - loss: 8.1388e-05 - mse: 8.1388e-05 - 3s/epoch - 100ms/step
Epoch 10/10
26/26 - 3s - loss: 8.0373e-05 - mse: 8.0373e-05 - 3s/epoch - 100ms/step
9/9 [==============================] - 2s 41ms/step
RMSE is: 33.500256942658865
R2 is：0.9266279389679264
MAE is：19.7591290803196

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 234.26917099952698
Epoch 1/10
26/26 - 14s - loss: 9.2686e-05 - mse: 9.2686e-05 - 14s/epoch - 532ms/step
Epoch 2/10
26/26 - 3s - loss: 9.7512e-05 - mse: 9.7512e-05 - 3s/epoch - 98ms/step
Epoch 3/10
26/26 - 3s - loss: 1.3220e-04 - mse: 1.3220e-04 - 3s/epoch - 99ms/step
Epoch 4/10
26/26 - 3s - loss: 1.1260e-04 - mse: 1.1260e-04 - 3s/epoch - 100ms/step
Epoch 5/10
26/26 - 3s - loss: 1.0991e-04 - mse: 1.0991e-04 - 3s/epoch - 100ms/step
Epoch 6/10
26/26 - 3s - loss: 9.5005e-05 - mse: 9.5005e-05 - 3s/epoch - 100ms/step
Epoch 7/10
26/26 - 3s - loss: 9.8954e-05 - mse: 9.8954e-05 - 3s/epoch - 100ms/step
Epoch 8/10
26/26 - 3s - loss: 9.4819e-05 - mse: 9.4819e-05 - 3s/epoch - 100ms/step
Epoch 9/10
26/26 - 3s - loss: 8.8042e-05 - mse: 8.8042e-05 - 3s/epoch - 102ms/step
Epoch 10/10
26/26 - 3s - loss: 8.5654e-05 - mse: 8.5654e-05 - 3s/epoch - 100ms/step
9/9 [==============================] - 2s 39ms/step
RMSE is: 32.25564648749774
R2 is：0.9418737186084771
MAE is：19.173689557090

...

total training time(s): 862.7325768470764
Epoch 1/10
26/26 - 14s - loss: 5.3999e-05 - mse: 5.3999e-05 - 14s/epoch - 529ms/step
Epoch 2/10
26/26 - 3s - loss: 5.5201e-05 - mse: 5.5201e-05 - 3s/epoch - 99ms/step
Epoch 3/10
26/26 - 3s - loss: 5.0627e-05 - mse: 5.0627e-05 - 3s/epoch - 99ms/step
Epoch 4/10
26/26 - 3s - loss: 5.0803e-05 - mse: 5.0803e-05 - 3s/epoch - 99ms/step
Epoch 5/10
26/26 - 3s - loss: 4.9214e-05 - mse: 4.9214e-05 - 3s/epoch - 99ms/step
Epoch 6/10
26/26 - 3s - loss: 4.9076e-05 - mse: 4.9076e-05 - 3s/epoch - 102ms/step
Epoch 7/10
26/26 - 3s - loss: 4.9805e-05 - mse: 4.9805e-05 - 3s/epoch - 102ms/step
Epoch 8/10
26/26 - 3s - loss: 4.9107e-05 - mse: 4.9107e-05 - 3s/epoch - 100ms/step
Epoch 9/10
26/26 - 3s - loss: 5.0760e-05 - mse: 5.0760e-05 - 3s/epoch - 101ms/step
Epoch 10/10
26/26 - 3s - loss: 4.9020e-05 - mse: 4.9020e-05 - 3s/epoch - 101ms/step
9/9 [==============================] - 1s 32ms/step
RMSE is: 29.905548185436
R2 is：0.9475961336152641
MAE is：17.612352702153597
W

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 911.9049479961395
Epoch 1/10
26/26 - 14s - loss: 5.0178e-05 - mse: 5.0178e-05 - 14s/epoch - 527ms/step
Epoch 2/10
26/26 - 3s - loss: 4.6791e-05 - mse: 4.6791e-05 - 3s/epoch - 98ms/step
Epoch 3/10
26/26 - 3s - loss: 4.8950e-05 - mse: 4.8950e-05 - 3s/epoch - 99ms/step
Epoch 4/10
26/26 - 3s - loss: 5.1043e-05 - mse: 5.1043e-05 - 3s/epoch - 98ms/step
Epoch 5/10
26/26 - 3s - loss: 5.1672e-05 - mse: 5.1672e-05 - 3s/epoch - 99ms/step
Epoch 6/10
26/26 - 3s - loss: 5.0331e-05 - mse: 5.0331e-05 - 3s/epoch - 99ms/step
Epoch 7/10
26/26 - 3s - loss: 5.3154e-05 - mse: 5.3154e-05 - 3s/epoch - 100ms/step
Epoch 8/10
26/26 - 3s - loss: 4.9676e-05 - mse: 4.9676e-05 - 3s/epoch - 100ms/step
Epoch 9/10
26/26 - 3s - loss: 5.0276e-05 - mse: 5.0276e-05 - 3s/epoch - 99ms/step
Epoch 10/10
26/26 - 3s - loss: 5.3212e-05 - mse: 5.3212e-05 - 3s/epoch - 99ms/step
9/9 [==============================] - 2s 39ms/step
RMSE is: 32.27461560852078
R2 is：0.9433919882321389
MAE is：18.413145062982526
WM

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 975.4025094509125


# 15 **MINS**

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(2)
import numpy as np
np.set_printoptions(threshold=np.inf)
import time, os
import keras
keras.backend.set_image_data_format('channels_last')
from keras.layers import *
from keras.models import *
from tensorflow.keras.utils import plot_model
from keras.models import load_model
from keras.optimizers import Adam
from metrics import evaluate_performance


# os.chdir('D:/论文2/upload to GitHub/')
os.environ["PATH"] += os.pathsep + 'E:/Program Files (x86)/Graphviz2.38/bin' #used for visualizing the model

global_start_time = time.time()

def Unit(x, filters, pool=False):
	res = x
	if pool:
		x = MaxPooling2D(pool_size=(2, 2), padding="same")(x)
		res = Conv2D(filters=filters, kernel_size=[1, 1], strides=(2, 2), padding="same")(res)
	out = BatchNormalization()(x)
	out = Activation("relu")(out)
	out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

	out = BatchNormalization()(out)
	out = Activation("relu")(out)
	out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

	out = keras.layers.add([res, out])

	return out

def attention_3d_block(inputs,timesteps):
    #input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(timesteps, activation='linear')(a)
    a_probs = Permute((2, 1))(a)
    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

# Define the model
def multi_input_model(time_lag):
    """build multi input model构建多输入模型"""
    input1_ = Input(shape=(276, time_lag-1, 3), name='input1')
    input2_ = Input(shape=(276, time_lag-1, 3), name='input2')
    input3_ = Input(shape=(276, time_lag-1, 1), name='input3')
    input4_ = Input(shape=(11, time_lag-1, 1), name='input4')
    #first input
    x1 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input1_)
    x1 = Unit(x1, 32)
    x1 = Unit(x1, 64, pool=True)
    x1 = Flatten()(x1)
    x1 = Dense(276)(x1)

    # second input
    x2 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input2_)
    x2 = Unit(x2, 32)
    x2 = Unit(x2, 64, pool=True)
    x2 = Flatten()(x2)
    x2 = Dense(276)(x2)

    # third input
    x3 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input3_)
    x3 = Unit(x3, 32)
    x3 = Unit(x3, 64, pool=True)
    x3 = Flatten()(x3)
    x3 = Dense(276)(x3)

    # fourth input
    x4 = Flatten()(input4_)
    x4 = Dense(276)(x4)
    x4 = Reshape(target_shape=(276, 1))(x4)
    x4 = LSTM(128, return_sequences=True, input_shape=(276, 1))(x4)
    x4 = LSTM(276, return_sequences=False)(x4)
    x4 = Dense(276)(x4)

    out = keras.layers.add([x1, x2, x3, x4])
    out = Reshape(target_shape=(276, 1))(out)
    out = LSTM(128, return_sequences=True,input_shape=(276, 1))(out)
    out = attention_3d_block(out, 276)#shape of the output is（276，128）
    out = Flatten()(out)
    out = Dense(276)(out)

    model = Model(inputs=[input1_, input2_, input3_,input4_], outputs=[out]) #[input1_, input2_, input3_]
    return model

def build_model(X_train_1,X_train_2,X_train_3,X_train_4,Y_train,X_test_1,X_test_2,X_test_3,X_test_4,Y_test,\
	Y_test_original,batch_size,epochs,a,time_lag):

	X_train_1 = X_train_1.reshape(X_train_1.shape[0],  276, time_lag-1, 3)
	X_train_2 = X_train_2.reshape(X_train_2.shape[0],  276, time_lag-1, 3)
	X_train_3 = X_train_3.reshape(X_train_3.shape[0],  276, time_lag-1, 1)
	X_train_4 = X_train_4.reshape(X_train_4.shape[0],  11, time_lag-1, 1)
	Y_train = Y_train.reshape(Y_train.shape[0], 276)

	X_test_1 = X_test_1.reshape(X_test_1.shape[0],  276, time_lag-1, 3)
	X_test_2 = X_test_2.reshape(X_test_2.shape[0],  276, time_lag-1, 3)
	X_test_3 = X_test_3.reshape(X_test_3.shape[0],  276, time_lag-1, 1)
	X_test_4 = X_test_4.reshape(X_test_4.shape[0],  11, time_lag-1, 1)
	Y_test = Y_test.reshape(Y_test.shape[0], 276)

	if epochs == 50:
		model = multi_input_model(time_lag)
		model.compile(optimizer=Adam(), loss='mse', metrics=['mse'])
		model.fit([X_train_1, X_train_2, X_train_3, X_train_4], Y_train, batch_size=batch_size, epochs=epochs, verbose=2, shuffle=False)#, validation_split=0.05
		output = model.predict([X_test_1, X_test_2, X_test_3, X_test_4], batch_size=batch_size)
	else:
		# train models every 10 epoches
		model = load_model('testresult/'+str(epochs-10)+'-model-with-graph.h5')
		model.fit([X_train_1, X_train_2, X_train_3, X_train_4], Y_train, batch_size=batch_size, epochs=10, verbose=2, shuffle=False)# , validation_split=0.05
		output = model.predict([X_test_1, X_test_2, X_test_3, X_test_4], batch_size=batch_size)

	#rescale the output of this model将输出进行反归一化
	predictions = np.zeros((output.shape[0], output.shape[1]))
	for i in range(len(predictions)):
		for j in range(len(predictions[0])):
			predictions[i, j] = round(output[i, j]*a, 0)
			if predictions[i, j] < 0:
				predictions[i, j] = 0

	RMSE,R2,MAE,WMAPE=evaluate_performance(Y_test_original,predictions)
	#visualize the model structure
	plot_model(model, to_file='model.png', show_shapes=True)
	#print(model.summary())

	return model,Y_test_original,predictions,RMSE,R2,MAE,WMAPE

def Save_Data(path,model,Y_test_original,predictions,RMSE,R2,MAE,WMAPE,Run_epoch):
	print(Run_epoch)
	RMSE_ALL=[]
	R2_ALL=[]
	MAE_ALL=[]
	WMAPE_ALL=[]
	Average_train_time=[]
	RMSE_ALL.append(RMSE)
	R2_ALL.append(R2)
	MAE_ALL.append(MAE)
	WMAPE_ALL.append(WMAPE)
	model.save(path+str(Run_epoch)+'-model-with-graph.h5')
	np.savetxt(path+str(Run_epoch)+'-RMSE_ALL.txt', RMSE_ALL)
	np.savetxt(path+str(Run_epoch)+'-R2_ALL.txt', R2_ALL)
	np.savetxt(path+str(Run_epoch)+'-MAE_ALL.txt', MAE_ALL)
	np.savetxt(path+str(Run_epoch)+'-WMAPE_ALL.txt', WMAPE_ALL)
	with open(path+str(Run_epoch)+'-predictions.csv', 'w') as file:
		predictions = predictions.tolist()
		for i in range(len(predictions)):
			file.write(str(predictions[i]).replace("'", "").replace("[", "").replace("]", "")+"\n")
	with open(path+str(Run_epoch)+'-Y_test_original.csv', 'w') as file:
		Y_test_original = Y_test_original.tolist()
		for i in range(len(Y_test_original)):
			file.write(str(Y_test_original[i]).replace("'", "").replace("[", "").replace("]", "")+"\n")
	duration_time = time.time() - global_start_time
	Average_train_time.append(duration_time)
	np.savetxt(path+str(Run_epoch)+'-Average_train_time.txt', Average_train_time)
	print('total training time(s):', duration_time)

X_train_1,Y_train,X_test_1,Y_test,Y_test_original,a,b,X_train_2,X_test_2,X_train_3,X_test_3,X_train_4,X_test_4=\
	Get_All_Data(TG=15, time_lag=6, TG_in_one_day=72, forecast_day_number=5, TG_in_one_week=360)
Run_epoch = 50  # first training 50 epoch, and then add 10 epoch every time 初始训练epoch，以后每次加10，运行15次
for i in range(15):
	model, Y_test_original, predictions, RMSE, R2, MAE, WMAPE = build_model(X_train_1,X_train_2,X_train_3,X_train_4,Y_train,X_test_1,X_test_2,X_test_3,X_test_4,Y_test,\
		Y_test_original,batch_size=64,epochs=Run_epoch,a=a,time_lag=6)
	Save_Data("testresult/", model, Y_test_original, predictions, RMSE, R2, MAE, WMAPE, Run_epoch)
	Run_epoch += 10


#For Get_All_Data, change parameters referring to this: TG=15, time_lag=6, TG_in_one_day=72, forecast_day_number=5, TG_in_one_week=360
#10min:10,6,108,5,540,eopch=200
#15min:15,6,72,5,360 eopch=140
#30min:30,6,36,5,180 eopch=200
#60min:60,6,18,5,90 eopch=235

(1075, 276, 15) (1075, 276)
(355, 276, 15) (355, 276)
(355, 276)
4744 0
(1075, 276, 15)
(355, 276, 15)
(276, 276)
(1075, 276, 5)
(355, 276, 5)
(1075, 11, 5)
(355, 11, 5)
Epoch 1/50
17/17 - 20s - loss: 0.0808 - mse: 0.0808 - 20s/epoch - 1s/step
Epoch 2/50
17/17 - 2s - loss: 0.0060 - mse: 0.0060 - 2s/epoch - 101ms/step
Epoch 3/50
17/17 - 2s - loss: 0.0016 - mse: 0.0016 - 2s/epoch - 103ms/step
Epoch 4/50
17/17 - 2s - loss: 9.4062e-04 - mse: 9.4062e-04 - 2s/epoch - 102ms/step
Epoch 5/50
17/17 - 2s - loss: 4.0078e-04 - mse: 4.0078e-04 - 2s/epoch - 102ms/step
Epoch 6/50
17/17 - 2s - loss: 2.8815e-04 - mse: 2.8815e-04 - 2s/epoch - 102ms/step
Epoch 7/50
17/17 - 2s - loss: 2.3493e-04 - mse: 2.3493e-04 - 2s/epoch - 102ms/step
Epoch 8/50
17/17 - 2s - loss: 2.5377e-04 - mse: 2.5377e-04 - 2s/epoch - 101ms/step
Epoch 9/50
17/17 - 2s - loss: 2.3707e-04 - mse: 2.3707e-04 - 2s/epoch - 102ms/step
Epoch 10/50
17/17 - 2s - loss: 2.3984e-04 - mse: 2.3984e-04 - 2s/epoch - 103ms/step
Epoch 11/50
17/17 - 2s -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 187.1505789756775
Epoch 1/10
17/17 - 13s - loss: 8.4452e-05 - mse: 8.4452e-05 - 13s/epoch - 778ms/step
Epoch 2/10
17/17 - 2s - loss: 9.0823e-05 - mse: 9.0823e-05 - 2s/epoch - 102ms/step
Epoch 3/10
17/17 - 2s - loss: 9.6353e-05 - mse: 9.6353e-05 - 2s/epoch - 101ms/step
Epoch 4/10
17/17 - 2s - loss: 1.0129e-04 - mse: 1.0129e-04 - 2s/epoch - 102ms/step
Epoch 5/10
17/17 - 2s - loss: 1.0003e-04 - mse: 1.0003e-04 - 2s/epoch - 102ms/step
Epoch 6/10
17/17 - 2s - loss: 9.5295e-05 - mse: 9.5295e-05 - 2s/epoch - 102ms/step
Epoch 7/10
17/17 - 2s - loss: 1.0530e-04 - mse: 1.0530e-04 - 2s/epoch - 103ms/step
Epoch 8/10
17/17 - 2s - loss: 1.0176e-04 - mse: 1.0176e-04 - 2s/epoch - 105ms/step
Epoch 9/10
17/17 - 2s - loss: 8.6001e-05 - mse: 8.6001e-05 - 2s/epoch - 113ms/step
Epoch 10/10
17/17 - 2s - loss: 8.0788e-05 - mse: 8.0788e-05 - 2s/epoch - 112ms/step
6/6 [==============================] - 1s 44ms/step
RMSE is: 45.1902585887786
R2 is：0.9274428315962535
MAE is：27.559552969993

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 245.1948881149292
Epoch 1/10
17/17 - 15s - loss: 7.8282e-05 - mse: 7.8282e-05 - 15s/epoch - 877ms/step
Epoch 2/10
17/17 - 2s - loss: 7.8782e-05 - mse: 7.8782e-05 - 2s/epoch - 100ms/step
Epoch 3/10
17/17 - 2s - loss: 7.8929e-05 - mse: 7.8929e-05 - 2s/epoch - 101ms/step
Epoch 4/10
17/17 - 2s - loss: 8.3186e-05 - mse: 8.3186e-05 - 2s/epoch - 102ms/step
Epoch 5/10
17/17 - 2s - loss: 8.0745e-05 - mse: 8.0745e-05 - 2s/epoch - 102ms/step
Epoch 6/10
17/17 - 2s - loss: 7.6001e-05 - mse: 7.6001e-05 - 2s/epoch - 101ms/step
Epoch 7/10
17/17 - 2s - loss: 7.6193e-05 - mse: 7.6193e-05 - 2s/epoch - 101ms/step
Epoch 8/10
17/17 - 2s - loss: 7.4301e-05 - mse: 7.4301e-05 - 2s/epoch - 101ms/step
Epoch 9/10
17/17 - 2s - loss: 7.2001e-05 - mse: 7.2001e-05 - 2s/epoch - 102ms/step
Epoch 10/10
17/17 - 2s - loss: 7.2561e-05 - mse: 7.2561e-05 - 2s/epoch - 102ms/step
6/6 [==============================] - 2s 34ms/step
RMSE is: 41.09887050227249
R2 is：0.9340970497958708
MAE is：25.18606858542

...

total training time(s): 850.5036034584045
Epoch 1/10
17/17 - 12s - loss: 8.2907e-05 - mse: 8.2907e-05 - 12s/epoch - 733ms/step
Epoch 2/10
17/17 - 2s - loss: 7.6560e-05 - mse: 7.6560e-05 - 2s/epoch - 101ms/step
Epoch 3/10
17/17 - 2s - loss: 7.0577e-05 - mse: 7.0577e-05 - 2s/epoch - 101ms/step
Epoch 4/10
17/17 - 2s - loss: 7.8218e-05 - mse: 7.8218e-05 - 2s/epoch - 101ms/step
Epoch 5/10
17/17 - 2s - loss: 7.6244e-05 - mse: 7.6244e-05 - 2s/epoch - 102ms/step
Epoch 6/10
17/17 - 2s - loss: 8.0251e-05 - mse: 8.0251e-05 - 2s/epoch - 102ms/step
Epoch 7/10
17/17 - 2s - loss: 7.3873e-05 - mse: 7.3873e-05 - 2s/epoch - 104ms/step
Epoch 8/10
17/17 - 2s - loss: 6.9840e-05 - mse: 6.9840e-05 - 2s/epoch - 108ms/step
Epoch 9/10
17/17 - 2s - loss: 6.7729e-05 - mse: 6.7729e-05 - 2s/epoch - 114ms/step
Epoch 10/10
17/17 - 2s - loss: 6.3855e-05 - mse: 6.3855e-05 - 2s/epoch - 114ms/step
6/6 [==============================] - 1s 34ms/step
RMSE is: 50.50775558424658
R2 is：0.9297533468160541
MAE is：29.36578893651

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 899.3893074989319
Epoch 1/10
17/17 - 13s - loss: 5.9667e-05 - mse: 5.9667e-05 - 13s/epoch - 736ms/step
Epoch 2/10
17/17 - 2s - loss: 5.8304e-05 - mse: 5.8304e-05 - 2s/epoch - 100ms/step
Epoch 3/10
17/17 - 2s - loss: 5.7746e-05 - mse: 5.7746e-05 - 2s/epoch - 100ms/step
Epoch 4/10
17/17 - 2s - loss: 5.5963e-05 - mse: 5.5963e-05 - 2s/epoch - 100ms/step
Epoch 5/10
17/17 - 2s - loss: 5.4241e-05 - mse: 5.4241e-05 - 2s/epoch - 101ms/step
Epoch 6/10
17/17 - 2s - loss: 6.0700e-05 - mse: 6.0700e-05 - 2s/epoch - 101ms/step
Epoch 7/10
17/17 - 2s - loss: 5.8569e-05 - mse: 5.8569e-05 - 2s/epoch - 101ms/step
Epoch 8/10
17/17 - 2s - loss: 5.3486e-05 - mse: 5.3486e-05 - 2s/epoch - 102ms/step
Epoch 9/10
17/17 - 2s - loss: 5.6845e-05 - mse: 5.6845e-05 - 2s/epoch - 102ms/step
Epoch 10/10
17/17 - 2s - loss: 5.2291e-05 - mse: 5.2291e-05 - 2s/epoch - 101ms/step
6/6 [==============================] - 2s 51ms/step
RMSE is: 41.28068195900611
R2 is：0.9463578838382483
MAE is：24.06961624821

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 968.882383108139


# 30 **MINS**

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(2)
import numpy as np
np.set_printoptions(threshold=np.inf)
import time, os
import keras
keras.backend.set_image_data_format('channels_last')
from keras.layers import *
from keras.models import *
from tensorflow.keras.utils import plot_model
from keras.models import load_model
from keras.optimizers import Adam
from metrics import evaluate_performance


# os.chdir('D:/论文2/upload to GitHub/')
os.environ["PATH"] += os.pathsep + 'E:/Program Files (x86)/Graphviz2.38/bin' #used for visualizing the model

global_start_time = time.time()

def Unit(x, filters, pool=False):
	res = x
	if pool:
		x = MaxPooling2D(pool_size=(2, 2), padding="same")(x)
		res = Conv2D(filters=filters, kernel_size=[1, 1], strides=(2, 2), padding="same")(res)
	out = BatchNormalization()(x)
	out = Activation("relu")(out)
	out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

	out = BatchNormalization()(out)
	out = Activation("relu")(out)
	out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

	out = keras.layers.add([res, out])

	return out

def attention_3d_block(inputs,timesteps):
    #input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(timesteps, activation='linear')(a)
    a_probs = Permute((2, 1))(a)
    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

# Define the model
def multi_input_model(time_lag):
    """build multi input model构建多输入模型"""
    input1_ = Input(shape=(276, time_lag-1, 3), name='input1')
    input2_ = Input(shape=(276, time_lag-1, 3), name='input2')
    input3_ = Input(shape=(276, time_lag-1, 1), name='input3')
    input4_ = Input(shape=(11, time_lag-1, 1), name='input4')
    #first input
    x1 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input1_)
    x1 = Unit(x1, 32)
    x1 = Unit(x1, 64, pool=True)
    x1 = Flatten()(x1)
    x1 = Dense(276)(x1)

    # second input
    x2 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input2_)
    x2 = Unit(x2, 32)
    x2 = Unit(x2, 64, pool=True)
    x2 = Flatten()(x2)
    x2 = Dense(276)(x2)

    # third input
    x3 = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(input3_)
    x3 = Unit(x3, 32)
    x3 = Unit(x3, 64, pool=True)
    x3 = Flatten()(x3)
    x3 = Dense(276)(x3)

    # fourth input
    x4 = Flatten()(input4_)
    x4 = Dense(276)(x4)
    x4 = Reshape(target_shape=(276, 1))(x4)
    x4 = LSTM(128, return_sequences=True, input_shape=(276, 1))(x4)
    x4 = LSTM(276, return_sequences=False)(x4)
    x4 = Dense(276)(x4)

    out = keras.layers.add([x1, x2, x3, x4])
    out = Reshape(target_shape=(276, 1))(out)
    out = LSTM(128, return_sequences=True,input_shape=(276, 1))(out)
    out = attention_3d_block(out, 276)#shape of the output is（276，128）
    out = Flatten()(out)
    out = Dense(276)(out)

    model = Model(inputs=[input1_, input2_, input3_,input4_], outputs=[out]) #[input1_, input2_, input3_]
    return model

def build_model(X_train_1,X_train_2,X_train_3,X_train_4,Y_train,X_test_1,X_test_2,X_test_3,X_test_4,Y_test,\
	Y_test_original,batch_size,epochs,a,time_lag):

	X_train_1 = X_train_1.reshape(X_train_1.shape[0],  276, time_lag-1, 3)
	X_train_2 = X_train_2.reshape(X_train_2.shape[0],  276, time_lag-1, 3)
	X_train_3 = X_train_3.reshape(X_train_3.shape[0],  276, time_lag-1, 1)
	X_train_4 = X_train_4.reshape(X_train_4.shape[0],  11, time_lag-1, 1)
	Y_train = Y_train.reshape(Y_train.shape[0], 276)

	X_test_1 = X_test_1.reshape(X_test_1.shape[0],  276, time_lag-1, 3)
	X_test_2 = X_test_2.reshape(X_test_2.shape[0],  276, time_lag-1, 3)
	X_test_3 = X_test_3.reshape(X_test_3.shape[0],  276, time_lag-1, 1)
	X_test_4 = X_test_4.reshape(X_test_4.shape[0],  11, time_lag-1, 1)
	Y_test = Y_test.reshape(Y_test.shape[0], 276)

	if epochs == 50:
		model = multi_input_model(time_lag)
		model.compile(optimizer=Adam(), loss='mse', metrics=['mse'])
		model.fit([X_train_1, X_train_2, X_train_3, X_train_4], Y_train, batch_size=batch_size, epochs=epochs, verbose=2, shuffle=False)#, validation_split=0.05
		output = model.predict([X_test_1, X_test_2, X_test_3, X_test_4], batch_size=batch_size)
	else:
		# train models every 10 epoches
		model = load_model('testresult/'+str(epochs-10)+'-model-with-graph.h5')
		model.fit([X_train_1, X_train_2, X_train_3, X_train_4], Y_train, batch_size=batch_size, epochs=10, verbose=2, shuffle=False)# , validation_split=0.05
		output = model.predict([X_test_1, X_test_2, X_test_3, X_test_4], batch_size=batch_size)

	#rescale the output of this model将输出进行反归一化
	predictions = np.zeros((output.shape[0], output.shape[1]))
	for i in range(len(predictions)):
		for j in range(len(predictions[0])):
			predictions[i, j] = round(output[i, j]*a, 0)
			if predictions[i, j] < 0:
				predictions[i, j] = 0

	RMSE,R2,MAE,WMAPE=evaluate_performance(Y_test_original,predictions)
	#visualize the model structure
	plot_model(model, to_file='model.png', show_shapes=True)
	#print(model.summary())

	return model,Y_test_original,predictions,RMSE,R2,MAE,WMAPE

def Save_Data(path,model,Y_test_original,predictions,RMSE,R2,MAE,WMAPE,Run_epoch):
	print(Run_epoch)
	RMSE_ALL=[]
	R2_ALL=[]
	MAE_ALL=[]
	WMAPE_ALL=[]
	Average_train_time=[]
	RMSE_ALL.append(RMSE)
	R2_ALL.append(R2)
	MAE_ALL.append(MAE)
	WMAPE_ALL.append(WMAPE)
	model.save(path+str(Run_epoch)+'-model-with-graph.h5')
	np.savetxt(path+str(Run_epoch)+'-RMSE_ALL.txt', RMSE_ALL)
	np.savetxt(path+str(Run_epoch)+'-R2_ALL.txt', R2_ALL)
	np.savetxt(path+str(Run_epoch)+'-MAE_ALL.txt', MAE_ALL)
	np.savetxt(path+str(Run_epoch)+'-WMAPE_ALL.txt', WMAPE_ALL)
	with open(path+str(Run_epoch)+'-predictions.csv', 'w') as file:
		predictions = predictions.tolist()
		for i in range(len(predictions)):
			file.write(str(predictions[i]).replace("'", "").replace("[", "").replace("]", "")+"\n")
	with open(path+str(Run_epoch)+'-Y_test_original.csv', 'w') as file:
		Y_test_original = Y_test_original.tolist()
		for i in range(len(Y_test_original)):
			file.write(str(Y_test_original[i]).replace("'", "").replace("[", "").replace("]", "")+"\n")
	duration_time = time.time() - global_start_time
	Average_train_time.append(duration_time)
	np.savetxt(path+str(Run_epoch)+'-Average_train_time.txt', Average_train_time)
	print('total training time(s):', duration_time)

X_train_1,Y_train,X_test_1,Y_test,Y_test_original,a,b,X_train_2,X_test_2,X_train_3,X_test_3,X_train_4,X_test_4=\
	Get_All_Data(TG=30, time_lag=6, TG_in_one_day=36, forecast_day_number=5, TG_in_one_week=180)
Run_epoch = 50  # first training 50 epoch, and then add 10 epoch every time 初始训练epoch，以后每次加10，运行15次
for i in range(15):
	model, Y_test_original, predictions, RMSE, R2, MAE, WMAPE = build_model(X_train_1,X_train_2,X_train_3,X_train_4,Y_train,X_test_1,X_test_2,X_test_3,X_test_4,Y_test,\
		Y_test_original,batch_size=64,epochs=Run_epoch,a=a,time_lag=6)
	Save_Data("testresult/", model, Y_test_original, predictions, RMSE, R2, MAE, WMAPE, Run_epoch)
	Run_epoch += 10


#For Get_All_Data, change parameters referring to this: TG=15, time_lag=6, TG_in_one_day=72, forecast_day_number=5, TG_in_one_week=360
#10min:10,6,108,5,540,eopch=200
#15min:15,6,72,5,360 eopch=140
#30min:30,6,36,5,180 eopch=200
#60min:60,6,18,5,90 eopch=235

(535, 276, 15) (535, 276)
(175, 276, 15) (175, 276)
(175, 276)
8940 0
(535, 276, 15)
(175, 276, 15)
(276, 276)
(535, 276, 5)
(175, 276, 5)
(535, 11, 5)
(175, 11, 5)
Epoch 1/50
9/9 - 25s - loss: 0.0497 - mse: 0.0497 - 25s/epoch - 3s/step
Epoch 2/50
9/9 - 1s - loss: 0.0151 - mse: 0.0151 - 867ms/epoch - 96ms/step
Epoch 3/50
9/9 - 1s - loss: 0.0033 - mse: 0.0033 - 868ms/epoch - 96ms/step
Epoch 4/50
9/9 - 1s - loss: 0.0015 - mse: 0.0015 - 864ms/epoch - 96ms/step
Epoch 5/50
9/9 - 1s - loss: 0.0018 - mse: 0.0018 - 869ms/epoch - 97ms/step
Epoch 6/50
9/9 - 1s - loss: 8.3607e-04 - mse: 8.3607e-04 - 871ms/epoch - 97ms/step
Epoch 7/50
9/9 - 1s - loss: 3.5672e-04 - mse: 3.5672e-04 - 878ms/epoch - 98ms/step
Epoch 8/50
9/9 - 1s - loss: 2.4230e-04 - mse: 2.4230e-04 - 868ms/epoch - 96ms/step
Epoch 9/50
9/9 - 1s - loss: 1.8065e-04 - mse: 1.8065e-04 - 877ms/epoch - 97ms/step
Epoch 10/50
9/9 - 1s - loss: 1.6913e-04 - mse: 1.6913e-04 - 945ms/epoch - 105ms/step
Epoch 11/50
9/9 - 1s - loss: 1.4841e-04 - mse:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 107.45576453208923
Epoch 1/10
9/9 - 14s - loss: 5.1969e-05 - mse: 5.1969e-05 - 14s/epoch - 2s/step
Epoch 2/10
9/9 - 1s - loss: 5.2653e-05 - mse: 5.2653e-05 - 862ms/epoch - 96ms/step
Epoch 3/10
9/9 - 1s - loss: 5.3770e-05 - mse: 5.3770e-05 - 869ms/epoch - 97ms/step
Epoch 4/10
9/9 - 1s - loss: 5.4845e-05 - mse: 5.4845e-05 - 868ms/epoch - 96ms/step
Epoch 5/10
9/9 - 1s - loss: 5.6638e-05 - mse: 5.6638e-05 - 865ms/epoch - 96ms/step
Epoch 6/10
9/9 - 1s - loss: 5.8725e-05 - mse: 5.8725e-05 - 865ms/epoch - 96ms/step
Epoch 7/10
9/9 - 1s - loss: 6.1107e-05 - mse: 6.1107e-05 - 866ms/epoch - 96ms/step
Epoch 8/10
9/9 - 1s - loss: 6.4335e-05 - mse: 6.4335e-05 - 867ms/epoch - 96ms/step
Epoch 9/10
9/9 - 1s - loss: 6.6986e-05 - mse: 6.6986e-05 - 871ms/epoch - 97ms/step
Epoch 10/10
9/9 - 1s - loss: 6.8002e-05 - mse: 6.8002e-05 - 867ms/epoch - 96ms/step
3/3 [==============================] - 3s 53ms/step
RMSE is: 380.8392013809616
R2 is：0.44248971650630226
MAE is：208.6393374741200

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 137.83486533164978
Epoch 1/10
9/9 - 13s - loss: 6.8865e-05 - mse: 6.8865e-05 - 13s/epoch - 1s/step
Epoch 2/10
9/9 - 1s - loss: 7.1152e-05 - mse: 7.1152e-05 - 870ms/epoch - 97ms/step
Epoch 3/10
9/9 - 1s - loss: 7.6630e-05 - mse: 7.6630e-05 - 868ms/epoch - 96ms/step
Epoch 4/10
9/9 - 1s - loss: 8.7561e-05 - mse: 8.7561e-05 - 885ms/epoch - 98ms/step
Epoch 5/10
9/9 - 1s - loss: 1.0258e-04 - mse: 1.0258e-04 - 866ms/epoch - 96ms/step
Epoch 6/10
9/9 - 1s - loss: 1.1730e-04 - mse: 1.1730e-04 - 889ms/epoch - 99ms/step
Epoch 7/10
9/9 - 1s - loss: 1.2387e-04 - mse: 1.2387e-04 - 893ms/epoch - 99ms/step
Epoch 8/10
9/9 - 1s - loss: 1.1469e-04 - mse: 1.1469e-04 - 907ms/epoch - 101ms/step
Epoch 9/10
9/9 - 1s - loss: 1.0830e-04 - mse: 1.0830e-04 - 904ms/epoch - 100ms/step
Epoch 10/10
9/9 - 1s - loss: 1.0714e-04 - mse: 1.0714e-04 - 906ms/epoch - 101ms/step
3/3 [==============================] - 2s 29ms/step
RMSE is: 206.74718500941376
R2 is：0.7581378479746733
MAE is：115.2436853002

...

total training time(s): 571.0572392940521
Epoch 1/10
9/9 - 13s - loss: 5.6239e-05 - mse: 5.6239e-05 - 13s/epoch - 1s/step
Epoch 2/10
9/9 - 1s - loss: 5.3750e-05 - mse: 5.3750e-05 - 881ms/epoch - 98ms/step
Epoch 3/10
9/9 - 1s - loss: 5.0580e-05 - mse: 5.0580e-05 - 887ms/epoch - 99ms/step
Epoch 4/10
9/9 - 1s - loss: 4.9731e-05 - mse: 4.9731e-05 - 887ms/epoch - 99ms/step
Epoch 5/10
9/9 - 1s - loss: 4.3556e-05 - mse: 4.3556e-05 - 891ms/epoch - 99ms/step
Epoch 6/10
9/9 - 1s - loss: 4.2679e-05 - mse: 4.2679e-05 - 880ms/epoch - 98ms/step
Epoch 7/10
9/9 - 1s - loss: 5.2239e-05 - mse: 5.2239e-05 - 891ms/epoch - 99ms/step
Epoch 8/10
9/9 - 1s - loss: 6.3029e-05 - mse: 6.3029e-05 - 878ms/epoch - 98ms/step
Epoch 9/10
9/9 - 1s - loss: 7.4272e-05 - mse: 7.4272e-05 - 887ms/epoch - 99ms/step
Epoch 10/10
9/9 - 1s - loss: 8.9524e-05 - mse: 8.9524e-05 - 902ms/epoch - 100ms/step
3/3 [==============================] - 1s 52ms/step
RMSE is: 77.21240996490131
R2 is：0.938789581693144
MAE is：47.300766045548656


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 608.3199789524078
Epoch 1/10
9/9 - 12s - loss: 9.0696e-05 - mse: 9.0696e-05 - 12s/epoch - 1s/step
Epoch 2/10
9/9 - 1s - loss: 7.7539e-05 - mse: 7.7539e-05 - 869ms/epoch - 97ms/step
Epoch 3/10
9/9 - 1s - loss: 5.9343e-05 - mse: 5.9343e-05 - 867ms/epoch - 96ms/step
Epoch 4/10
9/9 - 1s - loss: 4.4061e-05 - mse: 4.4061e-05 - 867ms/epoch - 96ms/step
Epoch 5/10
9/9 - 1s - loss: 3.4352e-05 - mse: 3.4352e-05 - 868ms/epoch - 96ms/step
Epoch 6/10
9/9 - 1s - loss: 2.9094e-05 - mse: 2.9094e-05 - 870ms/epoch - 97ms/step
Epoch 7/10
9/9 - 1s - loss: 2.5957e-05 - mse: 2.5957e-05 - 866ms/epoch - 96ms/step
Epoch 8/10
9/9 - 1s - loss: 2.3193e-05 - mse: 2.3193e-05 - 881ms/epoch - 98ms/step
Epoch 9/10
9/9 - 1s - loss: 2.3203e-05 - mse: 2.3203e-05 - 868ms/epoch - 96ms/step
Epoch 10/10
9/9 - 1s - loss: 2.3261e-05 - mse: 2.3261e-05 - 878ms/epoch - 98ms/step
3/3 [==============================] - 2s 56ms/step
RMSE is: 60.2335016456531
R2 is：0.9691082508253654
MAE is：34.01136645962733
WM

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


total training time(s): 640.9350979328156
